In [1]:
import sys
sys.path.append('../../aai/')
from reversi import Color
import os
import os.path as path
import pickle

from chainer import cuda
from reversi import Color
from reversi.board import init, stringify, put, Board, is_valid

## 棋譜パース　valid確認付き

- 棋譜中でスペース以前は最善手ではなく棋譜のバリエーションを持たすためのある程度ランダムな手でスペース囲碁は最善手 (passではない)

In [ ]:
import sys
sys.path.append('../../aai/')
from reversi import Color
import os
import os.path as path
import pickle

DATA_ROOT = '/mnt/share/aai_kikura'
kifu_list = os.listdir(path.join(DATA_ROOT, './kifu'))
kifu_list = list(map(lambda x: path.join(DATA_ROOT, './kifu', x), kifu_list))

SPACE = ' '
c2num = {c:i for i, c in enumerate('ABCDEFGH')}

def parse(line):
    cursor = 0
    color = [Color.Black]
    plies = []

    def append_ply(x, y, color=color):
        plies.append((color[0], x*8 + y))
        color[0] = Color.Black if color[0] == Color.White else Color.White

    while True:
        y = line[cursor]
        cursor += 1

        if y == SPACE:  
            if line[cursor] not in c2num:
                # EoK
                meta = line[cursor:]
                return (plies, int(meta.split()[0]))

            else:
#                 append_ply(0, -1)
                continue

        y  = c2num[y]
        try:
            x = int(line[cursor]) - 1
            if not 0 <= x < 8:
                return None
        except ValueError:
            return None
        cursor += 1

        append_ply(x, y)
    
count = 42
data_all = []
for kifu in kifu_list[42:]:
    kifu_data = []
    print(kifu)
    with open(kifu) as f:
        s = f.read().split('\n')
    
    for line in s[:-1]:
        t = parse(line)
        if t:
            kifu_data.append(t)
    
    res = []
    for plies, score in kifu_data:
        c = Color.Black
        b = init()
        true_plies = []
        for ply in plies:
            x = ply[1] // 8
            y = ply[1] % 8

            if is_valid(b, c, x, y):
                true_plies.append((c, ply[1]))
                b = put(b, c, x, y)
            else:
                true_plies.append((c, -1))
                c = 1 - c
                if is_valid(b, c, x, y):
                    true_plies.append((c, ply[1]))
                    b = put(b, c, x, y)
                else:
                    break

            c = 1 - c
        else:
            if (b[0].sum() - b[1].sum()) == score:
                res.append((true_plies, score))
    count += 1
    with open(path.join('/mnt/share/aai_fukuta/kifu_data/data.{}.pkl'.format(count)), 'wb') as f:
        pickle.dump(res, f)
#     data_all.extend(res)

# with open(path.join('/mnt/share/aai_fukuta/kifu_data/data.all.pkl'), 'wb') as f:
#     pickle.dump(data_all, f)

In [35]:
data_all = []
for file in os.listdir('/mnt/share/aai_fukuta/kifu_data'):
    with open('/mnt/share/aai_fukuta/kifu_data/' + file, 'rb') as f:
        data = pickle.load(f)
        data_all.extend([kifu[0] for kifu in data])

In [37]:
len(data_all)

3792274

In [38]:
with open('/mnt/share/aai_fukuta/test.pkl','wb') as f:
    pickle.dump(data_all[:30000], f)
with open('/mnt/share/aai_fukuta/train_small.pkl', 'wb') as f:
    pickle.dump(data_all[30000:330000], f)
with open('/mnt/share/aai_fukuta/train_small.pkl', 'wb') as f:
    pickle.dump(data_all[30000:], f)

## 確認対局

In [168]:
c = Color.Black
b = init()

plies, score = res[0]
for ply in plies:
#     x, y = find_ply(b, c)
    x, y = ply[1]//8, ply[1]%8
    if ply[1] == -1:
        print('{0}: pass'.format('Black' if c == Color.Black else 'White'))
        print('')
    else:
        if not is_valid(b, ply[0], x, y):
            print('are?')
            break
        b = put(b, c, x, y)
        print('{0}: ({1}, {2}) {3}'.format('Black' if c == Color.Black else 'White', x, y, ply[1]))
        print(stringify(b))
        print('')

    c = 1 - c

(b[0].sum() - b[1].sum()) == score

Black: (4, 5) 37
    0   1   2   3   4   5   6   7 
  ┌───┬───┬───┬───┬───┬───┬───┬───┐
0 │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
1 │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
2 │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
3 │   │   │   │ ○ │ ● │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
4 │   │   │   │ ● │ ● │ ● │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
5 │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
6 │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
7 │   │   │   │   │   │   │   │   │
  └───┴───┴───┴───┴───┴───┴───┴───┘

White: (5, 3) 43
    0   1   2   3   4   5   6   7 
  ┌───┬───┬───┬───┬───┬───┬───┬───┐
0 │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
1 │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
2 │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
3 │   │   │   │ ○ │ ● │   │   │

True

In [129]:
i

73